In [3]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


In [21]:
import duckdb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

con.close()
con = duckdb.connect('dev.duckdb', read_only=True)

query = "SELECT * FROM main.stg_base_bruta_acidentes WHERE numero_boletim = '2021-009100344-001'"
df = con.execute(query).fetchdf()

# Configura o pandas para mostrar todas as colunas ao imprimir o DataFrame
pd.set_option('display.max_columns', None)

# Agora imprime as 100 primeiras linhas com todas as colunas visíveis
print(df.head(100))


       numero_boletim data_hora_boletim     data_inclusao tipo_acidente  \
0  2021-009100344-001  21/02/2021 17:02  21/02/2021 18:17        H08002   

                                  desc_tipo_acidente  cod_tempo  \
0  CHOQUE MECANICO COM VITIMA                    ...          4   

        desc_tempo  cod_pavimento        pavimento  cod_regional  \
0  NUBLADO                      1  ASFALTO                    23   

     desc_regional   origem_boletim local_sinalizado  velocidade_permitida  \
0  NORTE            POLICIA MILITAR              NÃO                    60   

   coordenada_x  coordenada_y hora_informada indicador_fatalidade  valor_ups  \
0     613674.04    7805466.31            SIM                  SIM          0   

   descricao_ups data_alteracao_smsa  valor_ups_antiga descricao_ups_antiga  
0  NÃO INFORMADO          00/00/0000                 0        NÃO INFORMADO  


In [20]:
con.close()